## Modelo: **Gemma 2** (2B)
---

In [1]:
from langchain_community.llms import Ollama 
from langchain_community.embeddings import OllamaEmbeddings

MODEL = "gemma2:2b"

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

### Prompt template

In [2]:
from langchain.prompts import PromptTemplate

template = """
Eres un representatne de servicio al cliente, responde la pregunta basado en el contexto de abajo. 
Si no puedes responder la pregunta, responde "No lo se"

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

### Extraer texto en *markdown* del archivo PDF con [Marker](https://github.com/VikParuchuri/marker])

#### Desde la terminal:

- Para convertir un solo archivo:

    `marker_single /path/to/file.pdf /path/to/output/folder --batch_multiplier 2 --max_pages 10`

- Para convertir multiples archivos:

    `marker /path/to/input/folder /path/to/output/folder --workers 4 --max 10 --min_length 10000`
---

***Nota:*** *Recomendado utilizar GPU (CUDA o MPS) y 16 GB de RAM*

### Markdown document loader 

In [3]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

markdown_path = "docs/manual-lpcr/manual-lpcr.md" 
md_loader = UnstructuredMarkdownLoader(markdown_path)

md_pages = md_loader.load_and_split()

### Vector Store

In [4]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    md_pages, 
    embedding=embeddings
    )

# Instantiate a retriever
retriever = vectorstore.as_retriever()

/Users/ivanlee/llama-rag-demo/.venv/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


### Chain

In [5]:
from langchain_core.runnables import RunnablePassthrough

md_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
}
| prompt
| model
)

### Queries

In [6]:
questions = [
    "Como peso un producto?",
    "Como se introduce el precio por kilogramo?",
    "Como puedo borrar la venta total?",
]

for question in questions:
    print(f"Pregunta: {question}")
    print(f"Respuesta: {md_chain.invoke({'question': question})}")
    print()

Pregunta: Como peso un producto?
Respuesta: Here's how to weigh a product, based on the provided text:

1. **Position the product:** Place the product you want to weigh on the platform of the scale. 
2. **Turn on the scale:** Ensure the scale is switched on and ready for weighing.
3. **Observe the display:** The scale will typically show a "0" or a starting weight as soon as you place an item on it. Then, the digital display will update with the correct weight.  

**Important Notes:** 

* **Cleanliness:** Make sure to clean any spills or dirt from the platform before placing your product for accurate measurements.
* **Stability:** Ensure the product is positioned in a stable manner to avoid any fluctuations during the weighing process.


Let me know if you have any other questions! 


Pregunta: Como se introduce el precio por kilogramo?
Respuesta: The price per kilogram for the L-PCR-20 is **$999.99** and for the L-PCR-40 is **$999.99**. 


Pregunta: Como puedo borrar la venta total?
R